# Steps to solve Emergency vs Non-Emergency vehicle classification

<ol>1. Loading the dataset</ol>
<ol>2. Pre-processing the data</ol>
<ol>3. Creating training and validation set</ol>
<ol>4. Defining the model architecture</ol>
<ol>5. Compiling the model</ol>
<ol>6. Training the model</ol>
<ol><ol>Setting up model checkpointing</ol></ol>
<ol>7. Evaluating model performance</ol>

## 1. Loading the dataset

In [1]:
# import necessary libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# importing layers from keras
from keras.layers import Dense, InputLayer
from keras.models import Sequential
# importing adam optimizer from keras optimizer module 
from keras.optimizers import Adam

# train_test_split to create training and validation set
from sklearn.model_selection import train_test_split
# accuracy_score to calculate the accuracy of predictions
from sklearn.metrics import accuracy_score

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# unzipping the Dataset.zip file
# !unzip /content/drive/My\ Drive/Dataset.zip

In [3]:
# reading the csv file
data = pd.read_csv('../data/emergency_classification.csv')

In [4]:
# defining the seed value
seed = 42

In [5]:
# looking at first five rows of the data
data.head()

,image_names,emergency_or_not
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,1
4,4.jpg,1


In [6]:
# load images and store it in numpy array

# empty list to store the images
X = []
# iterating over each image
for img_name in data.image_names:
    # loading the image using its name
    img = plt.imread('../data/images_vehicles/' + img_name)
    # saving each image in the list
    X.append(img)
    
# converting the list of images into array
X=np.array(X)

# storing the target variable in separate variable
y = data.emergency_or_not.values

In [7]:
# shape of the images
X.shape

(2352, 224, 224, 3)

## 2. Pre-processing the data

In [8]:
# converting 3 dimensional image to 1 dimensional image
X = X.reshape(X.shape[0], 224*224*3)
X.shape

(2352, 150528)

In [9]:
# minimum and maximum pixel values of images
X.min(), X.max()

(0, 255)

In [10]:
# normalizing the pixel values
X = X / X.max()

In [11]:
# minimum and maximum pixel values of images after normalizing
X.min(), X.max()

(0.0, 1.0)

## 3. Creating training and validation set

In [12]:
# creating a training and validation set
X_train, X_valid, y_train, y_valid=train_test_split(X,y,test_size=0.3, random_state=seed)

In [13]:
# shape of training and validation set
(X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape)

(((1646, 150528), (1646,)), ((706, 150528), (706,)))

## 4. Defining the model architecture

In [14]:
# defining the model architecture
model=Sequential()

model.add(InputLayer(input_shape=(224*224*3,)))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

## 5. Compiling the model

In [15]:
# defining the adam optimizer and setting the learning rate as 10^-5
adam = Adam(lr=1e-5)

In [16]:
# compiling the model

# defining loss as binary crossentropy
# defining optimizer as Adam
# defining metrics as accuracy

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

## 6. Training the model

### Setting up model checkpointing

In [17]:
# importing model checkpointing from keras callbacks
from keras.callbacks import ModelCheckpoint

In [18]:
# defining model checkpointing

# defining the path to store the weights
filepath="best_weights.hdf5"

# defining the model checkpointing and metric to monitor
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# defining checkpointing variable
callbacks_list = [checkpoint]

In [20]:
# training the model for 50 epochs
model_history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_valid,y_valid), callbacks=callbacks_list)

Train on 1646 samples, validate on 706 samples
Epoch 1/50
1646/1646 [==============================] - 3s 2ms/step - loss: 0.6795 - accuracy: 0.5796 - val_loss: 0.6713 - val_accuracy: 0.5609

Epoch 00001: val_accuracy improved from -inf to 0.56091, saving model to best_weights.hdf5
Epoch 2/50
1646/1646 [==============================] - 2s 956us/step - loss: 0.6575 - accuracy: 0.5875 - val_loss: 0.6536 - val_accuracy: 0.5765

Epoch 00002: val_accuracy improved from 0.56091 to 0.57649, saving model to best_weights.hdf5
Epoch 3/50
1646/1646 [==============================] - 2s 954us/step - loss: 0.6426 - accuracy: 0.6033 - val_loss: 0.6429 - val_accuracy: 0.6105

Epoch 00003: val_accuracy improved from 0.57649 to 0.61048, saving model to best_weights.hdf5
Epoch 4/50
1646/1646 [==============================] - 2s 960us/step - loss: 0.6336 - accuracy: 0.6221 - val_loss: 0.6338 - val_accuracy: 0.6501

Epoch 00004: val_accuracy improved from 0.61048 to 0.65014, saving model to best_weights

## 8. Evaluating model performance 

In [21]:
# accuracy on validation set
print('Accuracy on validation set:', accuracy_score(y_valid, model.predict_classes(X_valid)[:, 0]), '%')

Accuracy on validation set: 0.7181303116147308 %


In [ ]:
# loading the best model
model.load_weights("best_weights.hdf5")

In [23]:
# accuracy on validation set
print('Accuracy on validation set:', accuracy_score(y_valid, model.predict_classes(X_valid)[:, 0]), '%')

Accuracy on validation set: 0.7237960339943342 %
